# Song Classification, Part 1

## This exercise comes from the Berkeley data science teaching materials

This exercise is a version of section 3, lab 4 from
<https://github.com/data-8/materials-x19> with thanks.  See that URL for more
on the license for these materials.  We release this version, like the
original, under the  [Creative Commons Attribution-NonCommercial 4.0
International license](https://creativecommons.org/licenses/by-nc/4.0) (CC
BY-NC 4.0).

## The song classifier

Over the next two labs, you will build your own song classifier using k-nearest
neighbors.

This is part 1 of the investigation.  Part 2 will follow soon.

You will build a classifier that guesses whether a song is hip-hop or country,
using only the numbers of times words appear in the song's lyrics.  By the end
of the project, you should know how to:

1. Clean and organize a dataset used to test a machine learning model
2. Build a k-nearest neighbors classifier
3. Test a classifier on data

**Advice.** Develop your answers incrementally. To perform a complicated table
manipulation, break it up into steps, perform each step on a different line,
give a new name to each result, and check that each intermediate result is what
you expect. You can add any additional names or functions you want to the
provided cells.

To get started, load our usual libraries.

In [ ]:
# Run this cell to set up the notebook, but please don't change it.
import numpy as np
import pandas as pd
# Safe settings for Pandas.
pd.set_option('mode.chained_assignment', 'raise')

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Load the OKpy test library and tests.
from client.api.notebook import Notebook
ok = Notebook('classify_song_1.ok')

## 1. The Dataset

Our dataset is a table of songs, each with a name, an artist, and a genre.
We'll be trying to predict each song's genre.

The only attributes we will use to predict the genre of a song are its lyrics.
In particular, we have a list of just under 5,000 words that might occur in a
song.  For each song, our dataset tells us the frequency with which each of
these words occurs in that song. All words have been converted to lowercase.

Run the cell below to read the `lyrics` table. **It may take a while to load.**

In [ ]:
lyrics = pd.read_csv('lyrics.csv')
# Show the first 6 rows
lyrics.head()

In [ ]:
# Show some interesting columns for "In Your Eyes"
cols = ['Title', 'Artist', 'Genre','i','the', 'you' , 'like', 'love']
lyrics.loc[lyrics['Title'] == 'In Your Eyes', cols]

That cell prints a few columns of the row for the country song ["In Your Eyes"
by Alison Krauss](http://www.azlyrics.com/lyrics/alisonkrauss/inyoureyes.html).
The song contains 168 words. The word "like" appears twice:  $\frac{2}{168}
\approx 0.0119$ of the words in the song. The word "love" appears 10 times:
$\frac{10}{168} \approx 0.0595$ of the words. The word "the" doesn't appear at
all.

Our dataset doesn't contain all information about a song.  For example, it
doesn't describe the order of words in the song, let alone the melody,
instruments, or rhythm. Nonetheless, you may find that word frequencies alone
are sufficient to build an accurate genre classifier.

All titles are unique.  To make it more convenient to fetch information about
songs by their title, we replace the original data frame with a new version
where the Pandas index (row labels) is (are) from the "Title" column.  For
later convenience, we will also leave the "Title" column in the data frame.  Of
course this means that the row label and the value of the "Title" column for
the row are always the same.

In [ ]:
# Run this cell.
# Set the row labels to be the titles.  Leave the titles as a column in the
# data frame.
lyrics = lyrics.set_index('Title', drop=False)
# Show the first six rows.
lyrics.head()

This makes it easier to get the song, using the title, by using `.loc` (label)
indexing on `lyrics`.

In [ ]:
lyrics.loc['In Your Eyes']

For example, the fastest way to find the frequency of "love" in the song *In
Your Eyes* is to access the `'love'` column from its row, like this:

In [ ]:
lyrics.loc['In Your Eyes', 'love']

### Question 1.1

Set `expected_row_sum` to the number that you expect will result from summing
all proportions in each row of `lyrics`, excluding the first three columns.

In [ ]:
#- Set row_sum to a number that's the (approximate) sum of each row of word
#- proportions.
expected_row_sum = ...

In [ ]:
_ = ok.grade('q1_1')

Run the cell below to generate a histogram of the actual row sums. It should
confirm your answer above, perhaps with a small amount of error.

In [ ]:
# Run this cell to display a histogram of the sums of proportions in each row.
# This computation might be a little slow.
# 'i' is the first word frequency column.
lyrics.loc[:, 'i':].sum(axis='columns').plot.hist()
# Matplotlib will show x values offset from 1 by default - fix.
plt.ticklabel_format(style='plain', useOffset=False)
plt.xticks(rotation=45);

This dataset was extracted from the [Million Song
Dataset](http://labrosa.ee.columbia.edu/millionsong/). Specifically, we are
using the complementary datasets from
[musiXmatch](http://labrosa.ee.columbia.edu/millionsong/musixmatch) and
[Last.fm](http://labrosa.ee.columbia.edu/millionsong/lastfm).

The counts of common words in the lyrics for all of these songs are provided by
the musiXmatch dataset (called a bag-of-words format). We converted the words
to lowercase, removed the naughty ones, and converted the counts to
frequencies.

The Last.fm dataset contains multiple tags for each song in the Million Song
Dataset. Some of the tags are genre-related, such as "pop", "rock", "classic",
etc.  To construct the `Genre` column, we first extracted songs with Last.fm
tags that included the words "country", or both "hip" and "hop". These songs
were then cross-referenced with the musiXmatch dataset, and only songs with
musixMatch lyrics were placed into our dataset.

The number of songs in the data frame is just the number of rows in the data
frame.

In [ ]:
# Run this cell.
print('Songs with genres:', len(lyrics))

The number of words for which we have frequencies, is the number of columns,
after the first three non-frequency columns, "Title", "Artist" and "Genre".

In [ ]:
# Run this cell.
# Get the column names as a list, throw away the first three.
word_col_names = list(lyrics)[3:]
print('Words with frequencies:', len(word_col_names))

## 1.1. Word Stemming

The columns other than "Title", "Artist", and "Genre" in the `lyrics`
data frame are all words that appear in some of the songs in our dataset.  Some
of those names have been *stemmed*, or abbreviated heuristically, in an attempt
to make different [inflected](https://en.wikipedia.org/wiki/Inflection) forms
of the same base word into the same string.  For example, the column "manag" is
the sum of proportions of the words "manage", "manager", "managed", and
"managerial" (and perhaps others) in each song.

Stemming makes it a little tricky to search for the words you want to use, so
we have provided another table that will let you see examples of unstemmed
versions of each stemmed word.  Run the code below to load it.

In [ ]:
# Just run this cell.
# Get data frame giving stem and word pairs.
vocab_mapping = pd.read_csv('mxm_reverse_mapping_safe.csv')
# Make a table for the word stems in the lyrics data frame.
stems = pd.DataFrame()
stems['Stem'] = sorted(word_col_names)
# Look up the lyrics column names in the vocab_mapping data frame.
vocab_table = stems.merge(vocab_mapping, on='Stem')
# Show some example rows
vocab_table.iloc[np.arange(1100, 1106), :]

### Question 1.1.1

Assign `unchanged` to the **percentage** of words in `vocab_table` that are the
same as their stemmed form (such as "devour" above).

*Hint:*  You might try comparing the columns, and then using `count_nonzero`,
on the way to a solution.

In [ ]:
percent_unchanged = ...
print(round(percent_unchanged, 2), 'percent are unchanged')

In [ ]:
_ = ok.grade("q1_1_1")

### Question 1.1.2

Assign `stemmed_message` to the stemmed version of the word "message".

*Hint*: the stemmed version of the word is in the "Stem" column of
`vocab_table`.

In [ ]:
#- Set stemmed_message to the stemmed version of "message" (which
#- should be a string).  Use vocab_table.
stemmed_message = ...
stemmed_message

In [ ]:
_ = ok.grade("q1_1_2")

### Question 1.1.3

Assign `unstemmed_singl` to the word in `vocab_table` that has "singl" as its
stemmed form. (*Note that multiple English words may stem to "singl", but only
one example appears in `vocab_table`.*)

In [ ]:
#- Set unstemmed_singl to the unstemmed version of "singl" (which
#- should be a string).
unstemmed_singl = ...
unstemmed_singl

In [ ]:
_ = ok.grade('q1_1_3')

## 1.2. Splitting the dataset

We're going to use our `lyrics` dataset for two purposes.

1. First, we want to *train* song genre classifiers.
2. Second, we want to *test* the performance of our classifiers.

Hence, we need two different datasets: *training* and *test*.

The purpose of a classifier is to classify unseen data that is similar to the
training data. Therefore, we must ensure that there are no songs that appear in
both sets. We do so by splitting the dataset randomly. The dataset has already
been permuted randomly, so it's easy to split.  We just take the top for
training and the rest for test.

Run the code below (without changing it) to separate the datasets into two
tables.

In [ ]:
# Here we have defined the proportion of our data
# that we want to designate for training as 11/16ths
# of our total dataset.  5/16ths of the data is
# reserved for testing.

training_proportion = 11/16

num_songs = len(lyrics)
num_train = int(num_songs * training_proportion)
num_valid = num_songs - num_train

train_lyrics = lyrics.iloc[:num_train]
test_lyrics = lyrics.iloc[num_train:]

print("Training: ",   len(train_lyrics), ";",
      "Test: ",       len(test_lyrics))

Run the code below to display a horizontal bar chart with two bars that show
the proportion of Country songs in each dataset.  We use the function
`country_proportion` to help us create the bar chart.

In [ ]:
def country_proportion(df):
    """Return the proportion of songs in a data frame of Country genre.
    """
    genre_counts = df['Genre'].value_counts()
    return genre_counts['Country'] / len(df)

plt.barh(['Test', 'Training'],
         [country_proportion(test_lyrics), country_proportion(train_lyrics)]);

## 2. K-Nearest Neighbors - a Guided Example

K-Nearest Neighbors (k-NN) is a classification algorithm.  Given some
*attributes* (also called *features*) of an unseen example, it decides whether
that example belongs to one or the other of two categories based on its
similarity to previously seen examples. Predicting the category of an example
is called *labeling*, and the predicted category is also called a *label*.

An attribute (feature) we have about each song is *the proportion of times a
particular word appears in the lyrics*, and the labels are two music genres:
hip-hop and country.  The algorithm requires many previously seen examples for
which both the attributes and labels are known: that's the `train_lyrics`
table.

To build understanding, we're going to visualize the algorithm instead of just
describing it.


## 2.1. Classifying a  song

In k-NN, we classify a song by finding the `k` songs in the *training set* that
are most similar according to the features we choose. We call those songs with
similar features the *nearest neighbors*.  The k-NN algorithm assigns the song
to the most common category among its `k` nearest neighbors.

Let's limit ourselves to just 2 features for now, so we can plot each song.
The features we will use are the proportions of the words "like" and "love" in
the lyrics.  Taking the song "In Your Eyes" (in the test set), 0.0119 of its
words are "like" and 0.0595 are "love". This song appears in the test set, so
let's imagine that we don't yet know its genre.

First, we need to make our notion of similarity more precise.  We will say that
the *distance* between two songs is the straight-line distance between them
when we plot their features in a scatter diagram. This distance is called the
Euclidean ("yoo-KLID-ee-un") distance, whose formula is $\sqrt{(x_1 - x_2)^2 +
(y_1 - y_2)^2}$.

For example, in the song *Insane in the Brain* (in the training set), 0.0203 of
all the words in the song are "like" and 0 are "love".  Its distance from *In
Your Eyes* on this 2-word feature set is $\sqrt{(0.0119 - 0.0203)^2 + (0.0595 -
0)^2} \approx 0.06$.  (If we included more or different features, the distance
could be different.)

A third song, *Sangria Wine* (in the training set), is 0.0044 "like" and 0.0925
"love".

The function below creates a plot to display the "like" and "love" features of
a test song and some training songs. As you can see in the result, *In Your
Eyes* is more similar to *Sangria Wine* than to *Insane in the Brain*.

In [ ]:
# Just run this cell.

def genre_to_color(v):
    """ Return 'gold' for 'Country', 'darkblue' for 'Hip-hop', 'red' otherwise.
    """
    if v == 'Country':
        return 'gold'
    elif v == 'Hip-hop':
        return 'darkblue'
    return 'red'


def plot_with_two_features(test_song, training_songs, x_feature, y_feature):
    """Plot a test song and training songs using two features."""
    # Columns to fetch.
    cols = [x_feature, y_feature, 'Genre']
    distances = lyrics.loc[training_songs, cols].copy()
    # Append row for test row
    test_row = lyrics.loc[test_song, cols].copy()
    test_row['Genre'] = 'Unknown'
    distances = distances.append(test_row)
    distances['Color'] = distances['Genre'].apply(genre_to_color)
    distances.plot.scatter(x_feature, y_feature, c='Color', s=200)

training = ["Sangria Wine", "Insane In The Brain"]
plot_with_two_features("In Your Eyes", training, "like", "love")

### Question 2.1.1

Compute the distance between the two country songs, *In Your Eyes* and *Sangria
Wine*, using the `like` and `love` features only.  Assign it the name
`country_distance`.

**Note:** If you have a row Series, you can use `.loc` (label) indexing to get
a value from a column by its name.  For example, if `r` is a Series from a row,
then `r.loc["Genre"]` is the value in column `"Genre"` in row `r`.

**Note 2:** You can quickly get the row from the `lyrics` table using
`.loc` (label) indexing by title.  For example, `lyrics.loc["Insane In
The Brain"]` is the row Series for this song.

In [ ]:
in_your_eyes = lyrics.loc["In Your Eyes"]
sangria_wine = lyrics.loc["Sangria Wine"]
like_diff = ...
love_diff = ...
country_distance = ...
country_distance

In [ ]:
_ = ok.grade("q2_1_1")

The `plot_with_two_features` function can show the positions of several
training songs. Below, we've added one that's even closer to *In Your Eyes*.

In [ ]:
training = ["Sangria Wine", "Lookin' for Love", "Insane In The Brain"]
plot_with_two_features("In Your Eyes", training, "like", "love")

### Question 2.1.2

Complete the function `distance_two_features` that computes the Euclidean
distance between any two songs, using two features. The last two lines call
your function to show that *Lookin' for Love* is closer to *In Your Eyes* than
*Insane In The Brain*.

*Hint* Consider copying your code above, that works for the particular titles,
and particular features, and making the code work for any titles, and any
features.

In [ ]:
def distance_two_features(title0, title1, x_feature, y_feature):
    """Compute the distance between two songs with titles title0 and title1

    Only the features named x_feature and y_feature are used when computing the distance.
    """
    row0 = ...
    row1 = ...
    ...

for song in ["Lookin' for Love", "Insane In The Brain"]:
    song_distance = distance_two_features(song, "In Your Eyes", "like", "love")
    print(song, 'distance:\t', song_distance)

In [ ]:
_ = ok.grade("q2_1_2")

### Question 2.1.3

Define the function `distance_from_in_your_eyes` so that it works as described
in its documentation (the function "docstring").

*Hint*: Make use of your `distance_two_features` function.


In [ ]:
def distance_from_in_your_eyes(title):
    """Distance between the given song and "In Your Eyes", based on the features "like" and "love".

    Parameters
    ----------
    title: str
        A string, the name of a song.

    Returns
    -------
    distance : number
        Floating point number; distance between song with title `title`, and "In Your Eyes", using "like" and "love" as features.
    """
    ...

In [ ]:
# Try the function out on a test song.
# This should give the same answer you saw in the cell further above.
distance_from_in_your_eyes("Lookin' for Love")

In [ ]:
_ = ok.grade('q2_1_3')

### Question 2.1.4

Using the features `"like"` and `"love"`, what are the names and genres of the
7 songs in the **training set** closest to "In Your Eyes"?  To answer this
question, make a table named `close_songs` containing those 7 songs with
columns `"Title"`, `"Artist"`, `"Genre"`, `"like"`, and `"love"`, as well as a
column called `"distance"` that contains the distance from "In Your Eyes".  The
table should be **sorted in ascending order by `distance`**.

*Hint*: there are various ways of doing this.

*Hint*: One way of doing this involves using `apply` on the "Title" column of a
new data frame.

In [ ]:
train_songs = ...
close_songs = ...
close_songs

In [ ]:
_ = ok.grade("q2_1_4")

**Question 2.1.5**

Define the function `most_common` so that it works as described in its
docstring below.

*Hint* : one way of doing this involves `value_counts`.

In [ ]:
    ...
# Calling most_common on your table of 7 nearest neighbors classifies
# "In Your Eyes" as a country song, 4 votes to 3.
most_common('Genre', close_songs)

In [ ]:
_ = ok.grade("q2_1_5")

Congratulations are in order -- you've classified your first song!


## The end

You're finished with part 1! We'll pick up from here for part 2, when we finish
our song classifier.

Be sure to:

- **run all the tests** (the next cell has a shortcut for that).
- **Save and Checkpoint** from the `File` menu.

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]